<a href="https://colab.research.google.com/github/Celso0408/Sticking-coefficient/blob/main/Sticking_coefficient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MEAN FIELD APPROXIMATION

$\newcommand{\nt}{\emph{n}}
\newcommand{\ckd}{c_\mathbf{k}^{\dagger}}
\newcommand{\ck}{c_\mathbf{k}^{\phantom{\dagger}}}
\newcommand{\ek}{\epsilon_{\mathbf{k}}^{\phantom{\dagger}}}
\newcommand{\ed}{E_{\mathbf{d}}^{\phantom{\dagger}}}
\newcommand{\rself}{\Sigma_R^{\phantom{\dagger}}}
\newcommand{\rselfsup}[1]{\Sigma_R^{#1}}
\newcommand{\iself}{\Delta_{\mathbf{d}}}
\newcommand{\iselfsup}[1]{\Delta_{\mathbf{d}}^{#1}}
\newcommand{\rd}{\rho_{dd}^{\phantom{\dagger}}}
\newcommand{\ups}{\uparrow}
\newcommand{\dos}{\downarrow}
\newcommand{\expvl}[1]{\langle #1\rangle}
\newcommand{\cn}[1]{c_{#1}^{\phantom{\dagger}}}
\newcommand{\cd}[1]{c_{#1}^{\dagger}}
\newcommand{\fn}[1]{f_{#1}^{\phantom{\dagger}}}
\newcommand{\fd}[1]{f_{#1}^{\dagger}}
\newcommand{\red}[1]{\textcolor{red}{#1}}
\newcommand{\ncr}{\nonumber\\}
\newcommand{\rhs}{right-hand side}
\newcommand{\occ}[1]{n^{\phantom{a}}_{#1}}
\newcommand{\hc}{{\text{H. c.}}}
\newcommand{\bracket}[3]{\langle#1|\,#2\,|#3\rangle}
\newcommand{\avrg}[2]{\langle #1 \, #2 \rangle }
\newcommand{\varbar}[1]{\underbar#1}
\newcommand{\dotpa}[2]{\langle#1|#2\rangle}
\def\doubleunderline#1{\underline{\underline{ \mathbf{#1} }}}
$

 
$ H_{\text{BO}} = Q(z)+
  \sum_{k}\epsilon_kc_k^\dagger c_k+ \sum_\sigma\Bigg(\epsilon_a+U\expvl{\occ{a,-\sigma}}-W(z)\Big((2\expvl{\occ{a,-\sigma}}-1)\expvl{\occ{0}}-2\expvl{\cd{a,-\sigma}\fn{0,-\sigma}}\expvl{\fd{0,-\sigma}\cn{a,-\sigma}}\Big) 
  \Bigg)\occ{a\sigma}\ncr -W(z)(2\expvl{\occ{a\ups}}\expvl{\occ{a\dos}}-\expvl{\occ{a\ups}}-\expvl{\occ{a\dos}}+1)\occ{0}
  +\sum_{\sigma}\Big( \big(V(z)-W(z)(2\expvl{\occ{a,-\sigma}}-1)\expvl{\fd{0\sigma}\cn{a\sigma}}\big)\cn{a\sigma}\fd{0\sigma}+\hc\Big)$

   where  $ Q(z) \equiv -(4\expvl{\occ{a\ups}\occ{a\dos}}-\expvl{\occ{a\ups}}-\expvl{\occ{a\dos}})\expvl{\occ{0}}+\sum_\sigma(4\expvl{\occ{a,-\sigma}}-1)\expvl{\cd{a\sigma}\fn{0\sigma}}\expvl{\fd{0\sigma}\cn{a\sigma}}$


In [1]:
import numpy as np
import sys
import random
import pandas as pd
from scipy.linalg import eigh
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

def sum_negative_numbers(arr):
    """
    Sum all negative numbers in a one-dimensional NumPy array.

    Parameters:
    arr (numpy.ndarray): A one-dimensional NumPy array.

    Returns:
    int: Sum of all negative numbers in the array.
    """
    if not isinstance(arr, np.ndarray):
        raise ValueError("Input must be a one-dimensional NumPy array.")

    if len(arr.shape) != 1:
        raise ValueError("Input must be a one-dimensional NumPy array.")
    
    negative_numbers = arr[arr < 0]
    return np.sum(negative_numbers)

def last_negative_index(arr):
    """
    Find the index of the last negative number in a one-dimensional NumPy array.

    Parameters:
    arr (numpy.ndarray): A one-dimensional NumPy array.

    Returns:
    int: Index of the last negative number in the array, or None if no negative numbers are found.
    """
    if not isinstance(arr, np.ndarray):
        raise ValueError("Input must be a one-dimensional NumPy array.")

    if len(arr.shape) != 1:
        raise ValueError("Input must be a one-dimensional NumPy array.")

    negative_indices = np.where(arr < 0)[0]
    
    if len(negative_indices) == 0:
        return None

    return negative_indices[-1]


def tn(j, d_band, lamb):
    
    # Define the function for tn Bulla, Costi Rev_Mod eq (32)
    
    # t_n = (1 + lamb ** (-1.0)) * (1.0 - lamb**(-j-1))#/2.0
    # Alternative: Campo, Oliveira PRB 72, 104432 (2005), Eq. (45) (converges faster to lamb->1 limit)
    # t_n = (1 - lamb**(-1)) / log(lamb) * (1.0 - lamb**(-j-1))

    # t_n = t_n/(np.sqrt(1.0-lamb**(-2*j-1)))
    # t_n = t_n/(np.sqrt(1.0-lamb**(-2*j-3)))
    # t_n = d_band*t_n

    


    # t_n = ((lamb ** (-(j - 2) / 2.0)) * (1.0 - lamb ** (-((j - 2) + 1))) *((1.0 - lamb ** (-1)))) / (np.sqrt(1.0 - lamb ** (-(2 * (j - 2) + 3))) * np.sqrt(1.0 - lamb ** (-(2 * (j - 1) + 1))) * np.log(lamb))  # electronvolt unit
    # t_n = d_band*t_n

    t_n = d_band*lamb**(-j/2.0) # eV

    # Second alternative: Ferreira, Oliveira PRB 106, 075129 (2022), Eq. (43)
    # Yields tight-binding conduction-band spectrum, as opposed to linear
    # t_n = (d_band / 2.0) * lamb**(-(n+1./2.))

    return t_n/27.41  # (a.u)

def W_pot(nuc_pos):
    # Define the image-charge potential W(z) in (a.u)
    return 1.0/(4.0*(nuc_pos+1.4))

def V_pot(nuc_pos):
    # define the hybridization V(z) in (a.u)
    return 0.1*np.tanh(-(nuc_pos)**2.0) + 0.1

def build_hamiltonian(nuc_pos, N = 3, spin = 1, Ea = -9.0, u = 12.8, d_band = 4.1,\
                      lamb = 4.5, av_cadca_down = 0.1, av_cadca_up = 0.1,\
                      av_f0df0_up = 0.1, av_f0df0_down = 0.1, av_cadf0_down = 0.1, av_cadf0_up = 0.1):
  
  # Construct the Hamiltonian matrix using the expectation values
  N = 2*N + 2 # the first two matrix element refers to f_0 and c_a
  
  # if N % 2 == 1: # making N even split the fermi level into an equal number of bands
  #   N += 1

  ham_ander = np.zeros((N, N))
  Ea = Ea/27.21 #(a.u)
  u = u/27.21 #(a.u)
  W = W_pot(nuc_pos)
  V = V_pot(nuc_pos)

  for ii in range(N):
    for jj in range(N):

      if spin == 1: # spin up

        if ii == 0 and jj == 0:
          ham_ander[ii, jj] = Ea + u * av_cadca_down \
          - W * ((2.0 * av_cadca_down - 1.0) * (av_f0df0_up + av_f0df0_down) - 2.0 * av_cadf0_down ** 2.0)
        elif ii == 0 and jj == 1:
          ham_ander[ii, jj] = V - W * (-2.0 * av_cadca_down + 1.0) * av_cadf0_up
          ham_ander[jj, ii] = ham_ander[ii, jj]
        elif ii == 1 and jj == 1:
          ham_ander[ii, jj] = -W * (2.0 * av_cadca_up * av_cadca_down - (av_cadca_up + av_cadca_down) + 1.0)
        elif (ii and jj) > 0 and ii == jj + 1:
          ham_ander[ii, jj] = tn(ii-1, d_band, lamb)
          ham_ander[jj, ii] = ham_ander[ii, jj]
      
      elif spin == -1: # spin down
    
        if ii == 0 and jj == 0:
          ham_ander[ii, jj] = Ea + u * av_cadca_up \
          - W * ((2.0 * av_cadca_up - 1.0) * (av_f0df0_up + av_f0df0_down) - 2.0 * av_cadf0_up ** 2.0)
        elif ii == 0 and jj == 1:
          ham_ander[ii, jj] = V - W * (-2.0 * av_cadca_up + 1.0) * av_cadf0_down
          ham_ander[jj, ii] = ham_ander[ii, jj]
        elif ii == 1 and jj == 1:
          ham_ander[ii, jj] = -W * (2.0 * av_cadca_up * av_cadca_down - (av_cadca_up + av_cadca_down) + 1.0)
        elif (ii and jj) > 0 and ii == jj + 1:
          ham_ander[ii, jj] = tn(ii-1, d_band, lamb)
          ham_ander[jj, ii] = ham_ander[ii, jj]
        
  return ham_ander

def diagonalize(hamiltonian):
  
    # Diagonalize the Hamiltonian matrix and return the eigenvalues and eigenvectors
    #eigenvalues, eigenvectors = eigh(hamiltonian)

    eigenvalues, eigenvectors = np.linalg.eig(hamiltonian)
    sorted_indices = np.argsort(eigenvalues)
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    
    n_dim = eigenvectors.shape

    for nvect in range(n_dim[0]):
      if eigenvectors[1][nvect] < 0.:
        for count in range(n_dim[0]):
          eigenvectors[count][nvect] = -eigenvectors[count][nvect]
    
    return sorted_eigenvalues, sorted_eigenvectors        


def solve_self_consistently(npoints=100, Ea = -9.0, u = 12.8, d_band = 8.2, lamb = 4.5, n_arderson_chain = 3):
  # Properties output dictionary
  scf_mf = {}

  # Convergecy parameters
  max_iterations = 10000000
  tol_fac = 1e-5   
  drho = 0.001 # incremental density

  # nuclear coodinate in (a.u)
  z_min = 0.001 
  z_max = 10.0
  z_nuc = np.linspace(z_min, z_max, npoints)
  z_nuc = z_nuc.tolist()
  
  cadca_up_list = []
  cadca_down_list = []
  f0df0_up_list = []
  f0df0_down_list = []

  w_pot_list = [] 
  v_pot_list = []
  const_mf = []
  gs_energy = []

  eigen_dict ={}

  jz = 0

  for nuc_var in tqdm(z_nuc):

    # initial guess 
    cadca_up = 0.8 
    f0df0_up = 0.8
    cadf0_up = 0.8

    cadca_down = 0.1
    f0df0_down = 0.2
    cadf0_down = 0.2
    
    tot_energy = 0.0 

    for itr in range(max_iterations):
      old_cadca_up = cadca_up
      old_f0df0_up = f0df0_up
      old_cadf0_up = cadf0_up

      old_cadca_down = cadca_down
      old_f0df0_down = f0df0_down
      old_cadf0_down = cadf0_down   

      hamiltonian_up = build_hamiltonian(nuc_pos = nuc_var, N = n_arderson_chain, spin = 1, Ea = Ea, u = u,\
                      d_band = d_band, lamb = lamb, av_cadca_down = old_cadca_down, av_cadca_up = old_cadca_up,\
                      av_f0df0_up = old_f0df0_up, av_f0df0_down = old_f0df0_down, av_cadf0_down = old_cadf0_down,\
                      av_cadf0_up = old_cadf0_up)

      eigenvalues_up, eigenvectors_up = diagonalize(hamiltonian_up)

      hamiltonian_down = build_hamiltonian(nuc_pos = nuc_var, N = n_arderson_chain, spin = -1, Ea = Ea, u = u,\
                      d_band = d_band, lamb = lamb, av_cadca_down = old_cadca_down, av_cadca_up = old_cadca_up,\
                      av_f0df0_up = old_f0df0_up, av_f0df0_down = old_f0df0_down, av_cadf0_down = old_cadf0_down,\
                      av_cadf0_up = old_cadf0_up)
      
      eigenvalues_down, eigenvectors_down = diagonalize(hamiltonian_down)

      ''' number of occupied orbitals for both spins '''
      n_fermi_up = last_negative_index(eigenvalues_up)
      n_fermi_down = last_negative_index(eigenvalues_down)
      
      old_tot_energy = tot_energy
      
      # compute the total energy of the occupied orbitals
      #tot_energy = sum_negative_numbers(eigenvalues_up) + sum_negative_numbers(eigenvalues_down)
      tot_energy = np.sum(eigenvalues_up[0:n_fermi_up]) + np.sum(eigenvalues_down[0:n_fermi_down])

      # compute the densities of the occupied orbitals of Ca and f0

      cadca_up = drho*(np.dot(eigenvectors_up[0,0:n_fermi_up], eigenvectors_up[0,0:n_fermi_up])) + (1.0 - drho)*cadca_up
      f0df0_up = drho*(np.dot(eigenvectors_up[1,0:n_fermi_up], eigenvectors_up[1,0:n_fermi_up])) + (1.0 - drho)*f0df0_up
      cadf0_up = (np.dot(eigenvectors_up[0,0:n_fermi_up], eigenvectors_up[1,0:n_fermi_up]))

      cadca_down = drho*(np.dot(eigenvectors_down[0,0:n_fermi_down], eigenvectors_down[0,0:n_fermi_down])) + (1.0 - drho)*cadca_down
      f0df0_down = drho*(np.dot(eigenvectors_down[1,0:n_fermi_down], eigenvectors_down[1,0:n_fermi_down])) + (1.0 - drho)*f0df0_down
      cadf0_down = np.dot(eigenvectors_down[0,0:n_fermi_down], eigenvectors_down[1,0:n_fermi_down])

      
      if (np.abs(old_cadca_down - cadca_down) <= tol_fac and \
          np.abs(old_f0df0_down - f0df0_down) <= tol_fac and \
          np.abs(old_cadca_up - cadca_up) <= tol_fac and \
          np.abs(old_f0df0_up - f0df0_up) <= tol_fac and \
          np.abs(tot_energy - old_tot_energy) <= tol_fac):
          #print("Converged after {} iterations.".format(itr))
          break
      elif itr == max_iterations:
          print("Warning: Maximum number of iterations reached without convergence.")
          #print("Nuc position:", nuc_var)
          break

    cadca_up_list.append(cadca_up)
    cadca_down_list.append(cadca_down)
    f0df0_up_list.append(f0df0_up)
    f0df0_down_list.append(f0df0_down)

    # mf energy constant
    q1 = -2.0*(cadca_down*(f0df0_down + f0df0_up)-cadf0_down**2.0)*cadca_up \
    +2.0*(cadf0_up**2.0)*cadca_down - cadf0_up**2.0
    q2 = 2.0*cadca_down*cadf0_up**2.0 -(-2.0*cadca_up +1.0)*cadf0_down**2.0\
    -(2.0*cadca_up*cadca_down-(cadca_up + cadca_down))*(f0df0_down + f0df0_up)

    const_mf.append(-(u/27.21)*cadca_up*cadca_down -W_pot(nuc_var)*(q1 + q2))
    tot_energy = tot_energy -(u/27.21)*cadca_up*cadca_down -W_pot(nuc_var)*(q1 + q2)
    
    gs_energy.append(tot_energy)
    w_pot_list.append(-W_pot(nuc_var))
    v_pot_list.append(V_pot(nuc_var))

    eigen_dict["jz_{}".format(jz)] = jz, eigenvalues_up, eigenvectors_up, eigenvalues_down, eigenvectors_down

    jz += 1
  
  # saving the properties
  
  scf_mf["nuc_cood"] = z_nuc 
  scf_mf["W_pot"] = w_pot_list
  scf_mf["V_pot"] = v_pot_list
  scf_mf["Const_mf"] = const_mf
  scf_mf["na_up"] = cadca_up_list
  scf_mf["na_down"] = cadca_down_list
  scf_mf["F0_up"] = f0df0_up_list
  scf_mf["F0_down"] = f0df0_down_list
  scf_mf["gs_energy"] = [x - gs_energy[-1] for x in gs_energy]

  return scf_mf, eigen_dict

# Main script
if __name__ == "__main__":
    
    # Physical parameters
    u_coulomb = 12.8 # (eV)
    ea_energy = -9.0 # (eV)
    d_band = 4.1     # (eV)
    n_chain = 5      # Wilson chain
    lamb = 4.5       # lambda
    npoints = 100    # Grid's number of points

    scf_mf, eigen_dict = solve_self_consistently(npoints = npoints, Ea = ea_energy, u = u_coulomb, d_band = d_band, lamb = lamb, n_arderson_chain = n_chain)


    # convert the dictionary to a DataFrame and display the head
    df = pd.DataFrame(list(scf_mf.items()), columns=['key', 'value'])
    #print(df.head(10))


  0%|          | 0/100 [00:00<?, ?it/s]

# Potentials and GS

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# create the subplots
fig = make_subplots(rows=3, cols=1, shared_xaxes=True,vertical_spacing=0.02)

# add data to the subplots
fig.add_trace(go.Scatter(x = scf_mf["nuc_cood"], y = scf_mf["W_pot"], mode='lines+markers', name='Image potential'), row=1, col=1)
fig.add_trace(go.Scatter(x = scf_mf["nuc_cood"], y = scf_mf["V_pot"], mode='lines+markers', name='Hybridization potential'), row=1, col=1)

fig.add_trace(go.Scatter(x = scf_mf["nuc_cood"], y = scf_mf["na_up"], mode='lines+markers', name=r'$\langle n_{a,\uparrow} \rangle$'), row=2, col=1)
fig.add_trace(go.Scatter(x = scf_mf["nuc_cood"], y = scf_mf["na_down"], mode='lines+markers', name=r'$\langle n_{a,\downarrow} \rangle$'), row=2, col=1)

fig.add_trace(go.Scatter(x = scf_mf["nuc_cood"], y = scf_mf["gs_energy"], mode='lines+markers', name='GS energy'), row=3, col=1)

# customize the layout
fig.update_layout(title='Time independent properties')

fig.update_yaxes(title_text='Potentials (a.u.)', row=1, col=1)
#fig.update_yaxes(title_text='V (a.u)', row=1, col=1)
fig.update_yaxes(title_text=r'$\langle n_{a,\sigma} \rangle$', row=2, col=1, range=[-0.1, 1.1])
fig.update_yaxes(title_text='Ground state (a.u.)', row=3, col=1)

# fig.update_xaxes(row=1, col=1, range=[-0.1, 10.1])
# fig.update_xaxes(row=2, col=1, range=[-0.1, 10.1])
fig.update_xaxes(title_text='z (a.u)', row=3, col=1, range=[-0.1, 10.1])
#fig.update_xaxes(title_text='na_down', row=4, col=1)

fig.update_layout(width=1400, height=700)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.23,
    xanchor="left",
    x=0.5
))

# show the figure
fig.show()

 

# Electron Hole pair Excitation

In [3]:
from itertools import permutations, product

def permute_list(input_list):
    first_element = input_list[0]
    second_element = input_list[1]
    remaining_list = input_list[2:]

    # Count the number of zeros and ones in the remaining list
    num_zeros = remaining_list.count(0)
    num_ones = remaining_list.count(1)

    # Create the list to permute
    permute_list = [0] * num_zeros + [1] * num_ones

    # Generate unique permutations
    unique_permutations = set(permutations(permute_list))

    # Add the fixed first two elements to each permutation
    result = [[first_element, second_element] + list(perm) for perm in unique_permutations]

    return result

def order_lists(input_list):
    # Calculate the value of the numbers multiplied by their index for each sublist
    def calculate_value(sublist):
        return sum(i * num for i, num in enumerate(sublist))

    # Sort the list of lists based on the calculated values
    sorted_list = sorted(input_list, key=calculate_value)
    
    return sorted_list

def zero_one_list(numbers):
    return [1 if num < 0 else 0 for num in numbers]

# up_energy = []
# down_energy = []
#number_of_excitation = 3

excitation_dict = {}
trunc_fac = 10 # truncating the number of excitations
state_dict = {}

for jz in tqdm(range(npoints)):
  list_up = eigen_dict[f"jz_{jz}"][1]
  list_down = eigen_dict[f"jz_{jz}"][3]

  binary_up = zero_one_list(list_up)
  binary_down = zero_one_list(list_down)

  result_up = permute_list(binary_up)
  result_down = permute_list(binary_down)

  sorted_list_up = order_lists(result_up)
  sorted_list_up = sorted_list_up[0:trunc_fac]
  sorted_list_down = order_lists(result_down)
  sorted_list_down = sorted_list_down[0:trunc_fac]

  prod_result = list(product(sorted_list_up, sorted_list_down))
  prod_temp = []

  for kk in range(trunc_fac**2):
    temp_list = []
    sorted_up = np.array(prod_result[kk][0])
    sorted_up.astype(np.float32)

    sorted_down = np.array(prod_result[kk][1])
    sorted_down.astype(np.float32)

    temp_up = np.dot(sorted_up, np.array(list_up))
    temp_down = np.dot(sorted_down, np.array(list_down))
    
    temp_tot = temp_up + temp_down + scf_mf["Const_mf"][jz]

    temp_list = list(sorted_up), list(sorted_down), temp_tot 
    prod_temp.append(temp_list)
  
  prod_temp = sorted(prod_temp, key=lambda x: x[-1])
  state_dict[f"up_and_down_{jz}"] = prod_temp

  0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
n_of_states = 80

states_list = []

for jj in range(n_of_states):
  temp_states = []
  for jz in range(npoints):
    temp_states.append(state_dict[f"up_and_down_{jz}"][jj][2])
  
  if jj == 0:
    fac_inft = temp_states[-1]
  
  temp_states = list((np.array(temp_states)- fac_inft)*27.21)
  
  states_list.append(temp_states)

# create the subplots
fig = make_subplots(rows=1, cols=1, shared_xaxes=True,vertical_spacing=0.02)

fig.add_trace(go.Scatter(x = scf_mf["nuc_cood"], y = states_list[0], mode='lines+markers', name='GS energy'), row=1, col=1)

for jj in range(1, n_of_states):
  fig.add_trace(go.Scatter(x = scf_mf["nuc_cood"], y = states_list[jj], mode='lines+markers', name=f"Energy_{jj}"), row=1, col=1)

# customize the layout
fig.update_layout(title='Time independent properties')

fig.update_yaxes(title_text='BO-PES (e.V)', row=1, col=1, range=[-4.1, 0.5])
#fig.update_yaxes(title_text='V (a.u)', row=1, col=1)
#fig.update_yaxes(title_text=r'$\langle n_{a,\sigma} \rangle$', row=2, col=1, range=[-0.1, 1.1])
#fig.update_yaxes(title_text='Ground state (a.u.)', row=3, col=1)

# fig.update_xaxes(row=1, col=1, range=[-0.1, 10.1])
# fig.update_xaxes(row=2, col=1, range=[-0.1, 10.1])
fig.update_xaxes(title_text='z (a.u)', row=3, col=1, range=[-0.1, 10.1])
fig.update_yaxes(title_text='z (a.u)', row=3, col=1, range=[-0.1, 10.1])
#fig.update_xaxes(title_text='na_down', row=4, col=1)

fig.update_layout(width=1400, height=700)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.23,
    xanchor="left",
    x=0.5
))

# show the figure
fig.show()